# 1. 爬虫准备
---
* 既然是数据的可视化，当然首先得获得这些数据，好在这个冠状病毒数据的爬虫的抓取并不困难。不过本人对爬虫研究不深，如有错误还望指正。
* 首先使用chrome打开腾讯的实时更新的新型肺炎最新动态的网页:https://news.qq.com/zt2020/page/feiyan.htm#/ ,然后按F12，按下图红色标注的地方依次点击，如果没有东西显示刷新网页即可。
* 之后就发现了有两个js:

    getOnsInfo?name=disease_h5&callback=jQuery341028144104595763175_1582955785410&_=1582955785411
    
    getOnsInfo?name=disease_other&callback=jQuery34109813744456178464_1582957974845&_=1582957974846


* 第一个记录的应该不同地区每天最新的时点数据，第二个包含了历史数据
* 任意点击其中一个，例如我点击了第一个，在右边通过preview可以看到它请求后得到的数据的形式，而在headers里就会发现它请求数据的url地址:
    
    https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5&callback=jQuery341028144104595763175_1582955785410&_=1582955785411
    
    https://view.inews.qq.com/g2/getOnsInfo?name=disease_other&callback=jQuery34109813744456178464_1582957974845&_=1582957974846
    
* 我开始不知道callback=jQuery341028144104595763175_1582955785410这样东西是可以去掉的，整的我获得的json格式的数据不能直接转换成字典。
* 1582955785411对应的应该是一个毫秒级别的时间戳。

In [1]:
import time
import json
import pandas as pd
import numpy as np

In [2]:
def get_last_data():
    import requests
    url = r"""https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5&_={0}""".format(int(time.time()*1000))
    result = requests.get(url)
    data = json.loads(result.json()['data'])
    
    return data

def get_hist_data():
    import requests
    url = r"""https://view.inews.qq.com/g2/getOnsInfo?name=disease_other&_={0}""".format(int(time.time()*1000))
    result = requests.get(url)
    data = json.loads(result.json()['data'])
    
    return data

In [23]:
last_data = get_last_data()

* 我们可以一层层分析data这个字典的结构，例如chinaTotal记录的是我国整体的情况，areaTree记录的是全国各省各地的情况
* 下面取两个我比较关注的上海和安徽来看

In [4]:
def get_city_last_info(data, province):
    for i, p in enumerate(data['areaTree'][0]['children']):
        if p['name'] == province:
            break
    today = list()
    total = list()
    
    for city in p['children']:
        city_today = city['today']
        city_today['city'] = city['name']
        city_today['province'] = province
        today.append(city_today)
        
    for city in p['children']:
        city_total = city['total']
        city_total['city'] = city['name']
        city_total['province'] = province
        total.append(city_total)
    
    return pd.DataFrame(today), pd.DataFrame(total)

In [5]:
zhejiang_today, zhejiang_total = get_city_last_info(last_data, '浙江')
anhui_today, anhui_total = get_city_last_info(last_data, '安徽')
hubei_today, hubei_total = get_city_last_info(last_data, '湖北')
shanghai_today, shanghai_total = get_city_last_info(last_data, '上海')

In [6]:
today = pd.concat((hubei_today, anhui_today, zhejiang_today, shanghai_today), axis=0)
today[today['province'] == '湖北']

,confirm,confirmCuts,isUpdated,city,province
0,0,0,False,境外输入,湖北
1,0,0,False,荆门,湖北
2,0,0,False,天门,湖北
3,0,0,False,宜昌,湖北
4,0,0,False,荆州,湖北
5,0,0,False,黄冈,湖北
6,0,0,False,十堰,湖北
7,0,0,False,孝感,湖北
8,0,0,False,黄石,湖北
9,0,0,False,仙桃,湖北


* 这里有两列，confirm我理解应该是新增确诊的意思，但是confirmCuts原谅我英语着急没整明白，都是0
* 在湖北的数据监测当中还加了监狱系统，但今天的数据好像没有更新

In [7]:
anhui_total

,nowConfirm,confirm,suspect,dead,deadRate,showRate,heal,healRate,showHeal,wzz,city,province,grade
0,0,3,0,0,0.00,False,3,100.00,True,0,境外输入,安徽,NaN
1,0,156,0,0,0.00,False,156,100.00,True,0,阜阳,安徽,全部低风险
2,0,174,0,1,0.57,False,173,99.43,True,0,合肥,安徽,NaN
3,0,6,0,0,0.00,False,6,100.00,True,0,宣城,安徽,NaN
4,0,17,0,0,0.00,False,17,100.00,True,0,池州,安徽,NaN
5,0,108,0,0,0.00,False,108,100.00,True,0,亳州,安徽,NaN
6,0,69,0,0,0.00,False,69,100.00,True,0,六安,安徽,NaN
7,0,41,0,0,0.00,False,41,100.00,True,0,宿州,安徽,NaN
8,0,9,0,0,0.00,False,9,100.00,True,0,黄山,安徽,NaN
9,0,83,0,0,0.00,False,83,100.00,True,0,安庆,安徽,NaN



* 再来看看总体的情况，以家乡安徽为例，合肥是确诊病例最高的，治愈人数也最多，蚌埠死亡病例最多(5例)，铜陵治愈率100%，29例患者全都恢复了健康
* 我大黄山还有2例没出院。不过我走之前就2例，起码一直没有新增，为家乡的防护措施点赞！
* 其实在notebook里面还有一些针对数据表格展示的工具，例如qgrid

# 2. 数据的可视化
---
* python绘图的库有很多，最最最常见的matplotlib，搭配seaborn即可制作精美的图表。
* matplotlib在notebook绘制的图表是静态的，没有发挥出notebook的强大，这里介绍两个常见的网页也是很常见的库echarts和highcharts。
* 安装: pip install pyecharts -i http://pypi.douban.com/simple --trusted-host pypi.douban.com和pip install python-highcharts

## 2.1 pyecharts
### 2.1.1 折线图

In [8]:
from pyecharts import options as opts
from pyecharts.globals import ThemeType
from pyecharts.charts import Line

In [28]:
hist_data = get_hist_data()
china_day_list = pd.DataFrame(hist_data['chinaDayList'])

In [10]:
china_day_list.head()

,confirm,suspect,dead,heal,nowConfirm,nowSevere,importedCase,deadRate,healRate,date,y,noInfect,localConfirm,noInfectH5,localConfirmH5
0,101187,4,4831,94298,2058,36,4772,4.8,93.2,02.04,2021,755,1008,473,1008
1,101241,2,4831,94447,1963,27,4780,4.8,93.3,02.05,2021,733,945,450,945
2,101272,3,4831,94597,1844,25,4790,4.8,93.4,02.06,2021,682,893,398,893
3,101316,3,4831,94729,1756,20,4804,4.8,93.5,02.07,2021,639,841,348,841
4,101363,2,4831,94839,1693,18,4818,4.8,93.6,02.08,2021,593,785,301,785


In [11]:
# 1. 创建折线图对象，并使用WALDENz主题
line = Line(init_opts=opts.InitOpts(theme=ThemeType.WALDEN))

# 2. 添加x轴数据
line.add_xaxis(china_day_list['date'])

# 3. 添加y数据
line.add_yaxis('现有确诊', china_day_list['nowConfirm'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False,
              linestyle_opts=opts.LineStyleOpts(width=2))
line.add_yaxis('累计确诊', china_day_list['confirm'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False, 
               linestyle_opts=opts.LineStyleOpts(width=2))
line.add_yaxis('疑似', china_day_list['suspect'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False, 
               linestyle_opts=opts.LineStyleOpts(width=2))

# 4. 全局配置
line.set_global_opts(
            title_opts=opts.TitleOpts(title="全国 现有确诊/疑似/累计确诊 趋势", subtitle="单位: 例"),
            yaxis_opts=opts.AxisOpts(axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(width=2, color="white")),
                                     splitline_opts=opts.SplitLineOpts(is_show=True)
                                    ),
            xaxis_opts=opts.AxisOpts(type_="category", axislabel_opts=opts.LabelOpts(rotate=45, font_family='Microsoft YaHei')),
            legend_opts=opts.LegendOpts(pos_right='right', pos_top='20%', legend_icon='circle', item_width=8),
            tooltip_opts=opts.TooltipOpts(is_show=True, trigger='axis', background_color='white', border_color='black', axis_pointer_type='line', 
                                          border_width=1,textstyle_opts=opts.TextStyleOpts(font_size=14, color='black')))
line.render_notebook()

In [12]:
line = Line(init_opts=opts.InitOpts(theme=ThemeType.WALDEN))

line.add_xaxis(china_day_list['date'])
line.add_yaxis('治愈', china_day_list['heal'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False,
              linestyle_opts=opts.LineStyleOpts(width=2))
line.add_yaxis('死亡', china_day_list['dead'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False, 
               linestyle_opts=opts.LineStyleOpts(width=2))
line.set_global_opts(
            title_opts=opts.TitleOpts(title="全国 累计治愈/死亡 趋势", subtitle="单位: 例"),
            yaxis_opts=opts.AxisOpts(axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(width=2, color="white")),
                                     splitline_opts=opts.SplitLineOpts(is_show=True)
                                    ),
            xaxis_opts=opts.AxisOpts(type_="category", axislabel_opts=opts.LabelOpts(rotate=45, font_family='Microsoft YaHei')),
            legend_opts=opts.LegendOpts(pos_right='right', pos_top='20%', legend_icon='circle', item_width=8),
            tooltip_opts=opts.TooltipOpts(is_show=True, trigger='axis', background_color='white', border_color='black', axis_pointer_type='line', 
                                          border_width=1,textstyle_opts=opts.TextStyleOpts(font_size=14, color='black')))
line.render_notebook()

In [13]:
china_day_add_list = pd.DataFrame(hist_data['chinaDayAddList'])

line = Line(init_opts=opts.InitOpts(theme=ThemeType.WALDEN))

line.add_xaxis(china_day_add_list['date'])
line.add_yaxis('新增治愈', china_day_add_list['heal'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False,
              linestyle_opts=opts.LineStyleOpts(width=2))
line.add_yaxis('新增确诊', china_day_add_list['confirm'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False, 
               linestyle_opts=opts.LineStyleOpts(width=2))
line.add_yaxis('新增死亡', china_day_add_list['dead'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False, 
               linestyle_opts=opts.LineStyleOpts(width=2))
line.add_yaxis('新增疑似', china_day_add_list['suspect'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False, 
               linestyle_opts=opts.LineStyleOpts(width=2))
line.set_global_opts(
            title_opts=opts.TitleOpts(title="全国 新增确诊/疑似/治愈/死亡 趋势", subtitle="单位: 例"),
            yaxis_opts=opts.AxisOpts(axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(width=2, color="white")),
                                     splitline_opts=opts.SplitLineOpts(is_show=True)
                                    ),
            xaxis_opts=opts.AxisOpts(type_="category", axislabel_opts=opts.LabelOpts(rotate=45, font_family='Microsoft YaHei')),
            legend_opts=opts.LegendOpts(pos_right='right', pos_top='20%', legend_icon='circle', item_width=8),
            tooltip_opts=opts.TooltipOpts(is_show=True, trigger='axis', background_color='white', border_color='black', axis_pointer_type='line', 
                                          border_width=1,textstyle_opts=opts.TextStyleOpts(font_size=14, color='black')))
line.render_notebook()

* 这里绘制了https://voice.baidu.com/act/newpneumonia/newpneumonia/?from=osari_pc_1 当中相应的图表。不难发现，这个网页的绘图插件使用的是echarts，借助pyecharts，我们在notebook里实现了90%类似的效果
* 好像这个网页的y轴做了处理，它的2月12号的新增数据看着不那么突兀
* 昨晚熬了夜，把pyecharts折线图基本的配置选项都在上面列了，读者可以手动尝试修改里面各项配置参数，看看不同的效果，可以更好的理解每个选项的含义

### 2.1.2 地图

In [14]:
from pyecharts import options as opts
from pyecharts.charts import Map

In [15]:
confirms = list()
for province in last_data['areaTree'][0]['children']:
    province_name = province['name']
    province_data = province['total']['confirm'] - province['total']['heal'] - province['total']['dead']
    confirms.append([province_name, province_data])

In [16]:
map_ = Map()
map_.add("地区", confirms, "china", is_map_symbol_show=False)
map_.set_global_opts(title_opts=opts.TitleOpts(title="当前现有确诊病例数，排除治愈、死亡"), 
                     visualmap_opts=opts.VisualMapOpts(max_=40000, is_piecewise=True,
                                                       pieces=[
                                                           {"min": 10000},
                                                           {"min": 3000, "max": 10000}, 
                                                           {"min": 1500, "max": 3000},
                                                           {"min": 800, "max": 1500},
                                                           {"min": 400, "max": 800},
                                                           {"min": 200, "max": 400},
                                                           {"min": 100, "max": 200},
                                                           {"min": 60, "max": 100},
                                                           {"min": 30, "max": 60},
                                                           {"min": 5, "max": 30},
                                                           {"max": 5}
                                                          ],
#                                                        range_color
                                                      ))
map_.render_notebook()

In [17]:
nameMap = {
            'Singapore Rep.':'新加坡',
            'Dominican Rep.':'多米尼加',
            'Palestine':'巴勒斯坦',
            'Bahamas':'巴哈马',
            'Timor-Leste':'东帝汶',
           'Afghanistan':'阿富汗',
           'Guinea-Bissau':'几内亚比绍',
           "Côte d'Ivoire":'科特迪瓦',
           'Siachen Glacier':'锡亚琴冰川',
           "Br. Indian Ocean Ter.":'英属印度洋领土',
           'Angola':'安哥拉',
           'Albania':'阿尔巴尼亚',
           'United Arab Emirates':'阿联酋',
           'Argentina':'阿根廷',
           'Armenia':'亚美尼亚',
           'French Southern and Antarctic Lands':'法属南半球和南极领地',
           'Australia':'澳大利亚',
           'Austria':'奥地利',
           'Azerbaijan':'阿塞拜疆',
           'Burundi':'布隆迪',
           'Belgium':'比利时',
           'Benin':'贝宁',
           'Burkina Faso':'布基纳法索',
           'Bangladesh':'孟加拉国',
           'Bulgaria':'保加利亚',
           'The Bahamas':'巴哈马',
           'Bosnia and Herz.':'波斯尼亚和黑塞哥维那',
           'Belarus':'白俄罗斯',
           'Belize':'伯利兹',
           'Bermuda':'百慕大',
           'Bolivia':'玻利维亚',
           'Brazil':'巴西',
           'Brunei':'文莱',
           'Bhutan':'不丹',
           'Botswana':'博茨瓦纳',
           'Central African Rep.':'中非',
           'Canada':'加拿大',
           'Switzerland':'瑞士',
           'Chile':'智利',
           'China':'中国',
           'Ivory Coast':'象牙海岸',
           'Cameroon':'喀麦隆',
           'Dem. Rep. Congo':'刚果民主共和国',
           'Congo':'刚果',
           'Colombia':'哥伦比亚',
           'Costa Rica':'哥斯达黎加',
           'Cuba':'古巴',
           'N. Cyprus':'北塞浦路斯',
           'Cyprus':'塞浦路斯',
           'Czech Rep.':'捷克',
           'Germany':'德国',
           'Djibouti':'吉布提',
           'Denmark':'丹麦',
           'Algeria':'阿尔及利亚',
           'Ecuador':'厄瓜多尔',
           'Egypt':'埃及',
           'Eritrea':'厄立特里亚',
           'Spain':'西班牙',
           'Estonia':'爱沙尼亚',
           'Ethiopia':'埃塞俄比亚',
           'Finland':'芬兰',
           'Fiji':'斐',
           'Falkland Islands':'福克兰群岛',
           'France':'法国',
           'Gabon':'加蓬',
           'United Kingdom':'英国',
           'Georgia':'格鲁吉亚',
           'Ghana':'加纳',
           'Guinea':'几内亚',
           'Gambia':'冈比亚',
           'Guinea Bissau':'几内亚比绍',
           'Eq. Guinea':'赤道几内亚',
           'Greece':'希腊',
           'Greenland':'格陵兰',
           'Guatemala':'危地马拉',
           'French Guiana':'法属圭亚那',
           'Guyana':'圭亚那',
           'Honduras':'洪都拉斯',
           'Croatia':'克罗地亚',
           'Haiti':'海地',
           'Hungary':'匈牙利',
           'Indonesia':'印度尼西亚',
           'India':'印度',
           'Ireland':'爱尔兰',
           'Iran':'伊朗',
           'Iraq':'伊拉克',
           'Iceland':'冰岛',
           'Israel':'以色列',
           'Italy':'意大利',
           'Jamaica':'牙买加',
           'Jordan':'约旦',
           'Japan':'日本',
           'Kazakhstan':'哈萨克斯坦',
           'Kenya':'肯尼亚',
           'Kyrgyzstan':'吉尔吉斯斯坦',
           'Cambodia':'柬埔寨',
           'Korea':'韩国',
           'Kosovo':'科索沃',
           'Kuwait':'科威特',
           'Lao PDR':'老挝',
           'Lebanon':'黎巴嫩',
           'Liberia':'利比里亚',
           'Libya':'利比亚',
           'Sri Lanka':'斯里兰卡',
           'Lesotho':'莱索托',
           'Lithuania':'立陶宛',
           'Luxembourg':'卢森堡',
           'Latvia':'拉脱维亚',
           'Morocco':'摩洛哥',
           'Moldova':'摩尔多瓦',
           'Madagascar':'马达加斯加',
           'Mexico':'墨西哥',
           'Macedonia':'北马其顿',
           'Mali':'马里',
           'Myanmar':'缅甸',
           'Montenegro':'黑山',
           'Mongolia':'蒙古',
           'Mozambique':'莫桑比克',
           'Mauritania':'毛里塔尼亚',
           'Malawi':'马拉维',
           'Malaysia':'马来西亚',
           'Namibia':'纳米比亚',
           'New Caledonia':'新喀里多尼亚',
           'Niger':'尼日尔',
           'Nigeria':'尼日利亚',
           'Nicaragua':'尼加拉瓜',
           'Netherlands':'荷兰',
           'Norway':'挪威',
           'Nepal':'尼泊尔',
           'New Zealand':'新西兰',
           'Oman':'阿曼',
           'Pakistan':'巴基斯坦',
           'Panama':'巴拿马',
           'Peru':'秘鲁',
           'Philippines':'菲律宾',
           'Papua New Guinea':'巴布亚新几内亚',
           'Poland':'波兰',
           'Puerto Rico':'波多黎各',
           'Dem. Rep. Korea':'朝鲜',
           'Portugal':'葡萄牙',
           'Paraguay':'巴拉圭',
           'Qatar':'卡塔尔',
           'Romania':'罗马尼亚',
           'Russia':'俄罗斯',
           'Rwanda':'卢旺达',
           'W. Sahara':'西撒哈拉',
           'Saudi Arabia':'沙特阿拉伯',
           'Sudan':'苏丹',
           'S. Sudan':'南苏丹',
           'Senegal':'塞内加尔',
           'Solomon Is.':'所罗门群岛',
           'Sierra Leone':'塞拉利昂',
           'El Salvador':'萨尔瓦多',
           'Somaliland':'索马里兰',
           'Somalia':'索马里',
           'Serbia':'塞尔维亚',
           'Suriname':'苏里南',
           'Slovakia':'斯洛伐克',
           'Slovenia':'斯洛文尼亚',
           'Sweden':'瑞典',
           'Swaziland':'斯威士兰',
           'Syria':'叙利亚',
           'Chad':'乍得',
           'Togo':'多哥',
           'Thailand':'泰国',
           'Tajikistan':'塔吉克斯坦',
           'Turkmenistan':'土库曼斯坦',
           'East Timor':'东帝汶',
           'Trinidad and Tobago':'特里尼达和多巴哥',
           'Tunisia':'突尼斯',
           'Turkey':'土耳其',
           'Tanzania':'坦桑尼亚',
           'Uganda':'乌干达',
           'Ukraine':'乌克兰',
           'Uruguay':'乌拉圭',
           'United States':'美国',
           'Uzbekistan':'乌兹别克斯坦',
           'Venezuela':'委内瑞拉',
           'Vietnam':'越南',
           'Vanuatu':'瓦努阿图',
           'West Bank':'西岸',
           'Yemen':'也门',
           'South Africa':'南非',
           'Zambia':'赞比亚',
           'Zimbabwe':'津巴布韦'
        }

In [18]:
name_map_ = {value: key for key, value in nameMap.items()}
name_map_['日本本土'] = 'Japan'

In [19]:
world_confirms = list()
for conuntry in last_data['areaTree']:
    try:
        conuntry_name = name_map_[conuntry['name']]
        conuntry_data = conuntry['total']['confirm'] - conuntry['total']['heal'] - conuntry['total']['dead']
        world_confirms.append([conuntry_name, conuntry_data])
    except Exception as e:
        print(conuntry['name'])

In [20]:
map_ = Map()
map_.add("国家", world_confirms, "world", is_map_symbol_show=False)
map_.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
map_.set_global_opts(title_opts=opts.TitleOpts(title="世界各国当前现有确诊病例数，排除治愈、死亡"), 
                     visualmap_opts=opts.VisualMapOpts(max_=40000, is_piecewise=True,
                                                       pieces=[
                                                           {"min": 10000},
                                                           {"min": 3000, "max": 10000}, 
                                                           {"min": 1500, "max": 3000},
                                                           {"min": 800, "max": 1500},
                                                           {"min": 400, "max": 800},
                                                           {"min": 200, "max": 400},
                                                           {"min": 100, "max": 200},
                                                           {"min": 60, "max": 100},
                                                           {"min": 30, "max": 60},
                                                           {"min": 5, "max": 30},
                                                           {"max": 5}
                                                          ],
#                                                        range_color
                                                      ))
map_.render_notebook()

## 2.2 highcharts
时间关系highcharts这里只做简单的介绍，建议在jupyter notebook当中交互式作图首先pyecharts，文档也更加丰富

In [21]:
from highcharts import Highchart
H = Highchart()

nowConfirm = china_day_list['nowConfirm'].tolist()
confirm = china_day_list['confirm'].tolist()
suspect = china_day_list['suspect'].tolist()

H.add_data_set(nowConfirm, 'line', '现有确诊')
H.add_data_set(confirm, 'line', '累计确诊')
H.add_data_set(suspect, 'line', '疑似')

H.set_options('chart', {'zoomType': 'x'})
H.set_options('xAxis', {'categories': china_day_list['date'].tolist()})
H.set_options('yAxis',{ 'title': { 'text': ''},
            'plotLines': {'value': 0, 'width': 1, 'color': '#808080'}})
H.set_options('tooltip', {'valueSuffix': '例', 'crosshairs': [{'width': 1, 'color': 'black'}] , 'shared': True})
H.set_options('title', {'text' : '全国 现有确诊/疑似/累计确诊 趋势'})
# H.set_options('line', {'dataLabels' : {'enable': False}})
H.set_options('legend', {'layout': 'vertical','align': 'right',
              'verticalAlign': 'middle','borderWidth': 0})
H.set_options('colors',{})
H.set_options('plotOptions',{'line': {
                'dataLabels': {
                    'enabled': False
                },
                'marker': {
                    'enabled': False
                                }
}})

H

ModuleNotFoundError: No module named 'highcharts'

# 3. 总结
* 本文通过爬虫抓取新冠肺炎相关数据，借助Python在notebook当中实现数据的可视化。
* 正所谓工欲善其事，必先利其器，借助jupyter notebook，我们可以轻松的实现代码，公式，图表的有机整合，实为我们研究的一大利器。
* 甚至我们可以举一反三，这个爬虫例子，可以进一步实现股票日行情数据的爬取。
* 最后也希望疫情早点结束，大家能够回归正常的生活！